In [1]:
!pip install dlt 

  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.5/767.5 kB 23.9 MB/s  0:00:00
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35/35 [dlt]32m34/35 [dlt]]lum]ormalizer]


In [2]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 29.0 MB/s  0:00:00m0:00:010:01


In [3]:
!pip install pandas 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 37.4 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 37.1 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pandas]2m1/2 [pandas]


In [4]:
!pip install python-dotenv 

In [5]:
import dlt 
import duckdb 
import pandas as pd 
import numpy as np 
from datetime import datetime 
import os 
from dotenv import load_dotenv 

load_dotenv()

print(f"Current working directory: {os.getcwd()}")

Current working directory: /workspaces/Duck_Data/notebook


In [6]:
pipeline = dlt.pipeline(
    pipeline_name="test_data_pipeline",
    destination="duckdb",
    dataset_name="my_data"    
)

print(f"Pipeline name: {pipeline.pipeline_name}")
print(f"Destination: {pipeline.destination}")
print(f"Dataset: {pipeline.dataset_name}")

Pipeline name: test_data_pipeline
Destination: <dlt.destinations.duckdb(destination_type='duckdb', staging_dataset_name_layout='%s_staging', enable_dataset_name_normalization=True, info_tables_query_threshold=1000, truncate_tables_on_staging_destination_before_load=True, local_dir='/workspaces/Duck_Data/notebook', pipeline_name='test_data_pipeline', pipeline_working_dir='/home/codespace/.dlt/pipelines/test_data_pipeline', create_indexes=False)>
Dataset: my_data


In [12]:
def load_transform(): 
    df = pd.read_csv("../Data_source/sample.csv")
    print(f"{len(df)} records loaded from CSV file.")
    print(f"Columns: {list(df.columns)}")
    
    print(f"Null values:{df.isnull().sum().sum()}")
    print(f"Duplicates rows: {df.duplicated().sum()}")
    print(f"Date range: from {df['date'].min()} to {df['date'].max()}")
    
    df['date'] = pd.to_datetime(df['date'])
    df['total_revenue'] = df['quantity'] * df['price']
    df['month']= df['date'].dt.month
    df['day of week'] = df['date'].dt.day_name()

    assert df['quantity'].min() > 0 #quantity must be positive 
    assert df ['price'].min() > 0 # price must be positive
    assert df['total_revenue'].min() > 0 # total_revenue must be positive
    
    print("\nData transformation completed.")

    for record in df.to_dict(orient='records'):
        yield record 
    
    

In [13]:
info = pipeline.run(
    load_transform(),
    table_name = 'sample_data',
    write_disposition = 'replace' #full load 
)

print(f"Load info:{info}")
print(f"Load Info Type:{type(info)}")

10 records loaded from CSV file.
Columns: ['date', 'product', 'category', 'quantity', 'price', 'region']
Null values:0
Duplicates rows: 0
Date range: from 2024-01-01 to 2024-01-05

Data transformation completed.
Load info:Pipeline test_data_pipeline load step completed in 1.15 seconds
1 load package(s) were loaded to destination duckdb and into dataset my_data
The duckdb destination used duckdb:////workspaces/Duck_Data/notebook/test_data_pipeline.duckdb location to store data
Load package 1754456070.2569757 is LOADED and contains no failed jobs
Load Info Type:<class 'dlt.common.pipeline.LoadInfo'>


In [15]:
duckdb_path = pipeline.sql_client().credentials.database
print(f"db path:{duckdb_path}")

db path:/workspaces/Duck_Data/notebook/test_data_pipeline.duckdb


In [24]:
con = duckdb.connect(duckdb_path) # connect to duckdb 

all_table = con.execute(""" 
    select table_schema, table_name
    from information_schema.tables
    where table_type = 'BASE TABLE' AND table_schema != 'information_schema'
""").fetchdf()

print(f"Avaolable tables: \n\n{all_table}")

Avaolable tables: 

  table_schema           table_name
0      my_data          sample_data
1      my_data           _dlt_loads
2      my_data  _dlt_pipeline_state
3      my_data         _dlt_version
